# Explorar a Camada Bronze

Analisar os dados na camada bronze para compreender o cenário e identificar as transformações necessárias para criar a camada Silver

## Importar bibliotecar

In [ ]:
import ipywidgets as widgets
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import dates
#
# Nome da aplicação Spark
#
APP_NAME="ExplorarBronze"

## Inicializar Spark e definir funções de apoio

In [ ]:
%run StartSpark.ipynb

## Carregar as tabelas da camada Bronze

In [ ]:
_ = loadAndRegister("stocks", "StockData", "bronze")

## Listar as tabelas e mostrar algumas informações sobre os dados

Lista de tabelas.

In [ ]:
%%sql_display
show tables

Quantidade de linhas por tipo de ação e tipo de operação

In [ ]:
%%sql_display
select 
    ticker,
    __op,
    count(*) as qtd
from bronze_stocks
group by all
order by ticker, __op

Quantidade de entradas por ação

In [ ]:
%%time
%%sql_display 
select 
    ticker,
    count(*) as qtd
from bronze_stocks
group by all
order by ticker

## Gerar um gráfico de variação no valor das ações

Primeiramente definimos a query

In [ ]:
df = %sql select ticker, timestamp - interval 3 hours as timestamp, close, volume from bronze_stocks where timestamp >= current_timestamp - interval 8 hours order by ticker, timestamp

Definição dos elementos da toolbar

In [ ]:
ticker = widgets.Dropdown(
    options=["AAPL", "AMZN", "GOOG", "MSFT", "TSLA", "NU"],
    value="AAPL",
    description="Stock:",
    disabled=False,
)

button = widgets.Button(
    description='Refresh',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Refresh Charts',
    icon='arrows-rotate' # (FontAwesome names without the `fa-` prefix)
)

Funções para plotar os gráficos

In [ ]:
def plotTickerValue(ticker):
    _df = df.toPandas()
    colors = sns.color_palette('pastel')[0:5]
    
    sns.set_style("ticks",{'axes.grid' : True})

    _, ax = plt.subplots()

    g = sns.lineplot(_df[_df.ticker == ticker], x="timestamp", y="close", ax=ax)
    _ = g.tick_params(axis="x", rotation=45) 
    _ = plt.title(f"Variação no Valor das Ações da {ticker}")
    _ = ax.xaxis.set_major_formatter(dates.DateFormatter("%y-%m-%d %H:%M"))

def plotTickerVolume(ticker):
    _df = df.toPandas()
    colors = sns.color_palette('pastel')[0:5]
    
    sns.set_style("ticks",{'axes.grid' : True})

    _, ax = plt.subplots()
    g = sns.lineplot(_df[_df.ticker == ticker], x="timestamp", y="volume", ax=ax)
    _ = g.tick_params(axis="x", rotation=45) 
    _ = plt.title(f"Variação no Volume Negociado da {ticker}")
    _ = ax.xaxis.set_major_formatter(dates.DateFormatter("%y-%m-%d %H:%M"))

### Renderizar a inteface definida

In [ ]:
toolbar = widgets.HBox([ticker, button])
w1 = widgets.interactive_output(plotTickerValue, {"ticker": ticker})
w2 = widgets.interactive_output(plotTickerVolume, {"ticker": ticker})
ui = widgets.HBox([w1, w2])
display(toolbar, ui)

def refresh(b):
    # plotTickerValue(ticker.value)
    # plotTickerVolume(ticker.value)
    display(toolbar, ui)

button.on_click(refresh)

## Preparar transformações para a camada Silver

Apresentar uma amostragem dos dados

In [ ]:
%%sql_display
select * 
from bronze_stocks
order by ticker, timestamp

### Transformações

Analisando os dados acima identificamos as seguintes transformações
1. **Particionamento de dados**: visando aumentar a performance, os dados serão particionados por _ticker_ (ação) e pelo dia (_timestamp_ formatado como YYYY-MM-DD). Para isso será adicionado uma coluna chamada _day_.
2. **Oscilação de valores**: baseando-se no campo _close_ (valor de fechamento do período atual), retirar seu valor do período anterior (função _lag_). Adicionando a coluna _osc_ com este valor.
3. **Percentual de Oscilação**: baseado no campo _osc_, criar uma coluna chamada _osc_per_ com o percentual da variação.
4. **Ajustar __timestamp__**: O campo *__ts_ms* que o horário em que o _kafka connector_ capturou o registro vem em formato _unix timestamp_ e precisa ser convertido para um formato mais amigável.

In [ ]:
df_silver = %sql \
SELECT \
    _id, \
    ticker, \
    date_format(timestamp, "yyyy-MM-dd") as day, \
    description, \
    timestamp, \
    open, \
    high, \
    low, \
    close, \
    volume,\
    (close - LAG(close,1) OVER (PARTITION BY ticker ORDER BY timestamp)) AS osc, \
    (osc * 100.0 / LAG(close,1) OVER (PARTITION BY ticker ORDER BY timestamp)) as osc_per, \
    __op, \
    __collection, \
    (to_timestamp(__ts_ms / 1000) - interval 5 hours) as __ts_ms \
from bronze_stocks \
where ticker = "AAPL"

In [ ]:
d = df_silver.fillna(value=0)

In [ ]:
d.toPandas()